In [1]:
from pyspark import SparkContext,SparkConf
from pyspark.sql import SQLContext, SparkSession, Row
from pyspark.sql.types import StructField, StringType, StructType, NumericType,IntegerType
# import pandas as pd
from datetime import datetime

conf = SparkConf().setAppName('APP_NAME')
conf = conf.setMaster("local[*]").set('spark.submit.deployMode', 'client')
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)
spark = SparkSession(sc)

In [2]:
# Download data
! cd data && wget http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz

converted 'http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz' (ANSI_X3.4-1968) -> 'http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz' (UTF-8)
--2020-02-09 18:35:27--  http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz
Resolving kdd.ics.uci.edu (kdd.ics.uci.edu)... 128.195.1.86
Connecting to kdd.ics.uci.edu (kdd.ics.uci.edu)|128.195.1.86|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2144903 (2.0M) [application/x-gzip]
Saving to: 'kddcup.data_10_percent.gz.1'

kddcup.data_10_perc 100%[=====================>]   2.04M   416KB/s   in 5.7s   

2020-02-09 18:35:34 (366 KB/s) - 'kddcup.data_10_percent.gz.1' saved [2144903/2144903]



In [6]:
data_file = "./data/kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file).cache()

csv_data = raw_data.map(lambda l: l.split(","))
row_data = csv_data.map(lambda p: Row(
    duration=int(p[0]), 
    protocol_type=p[1],
    service=p[2],
    flag=p[3],
    src_bytes=int(p[4]),
    dst_bytes=int(p[5])
    )
)

In [7]:
interactions_df = sqlContext.createDataFrame(row_data)
interactions_df.registerTempTable("interactions")

# Select tcp network interactions with more than 1 second duration and no transfer from destination
tcp_interactions = sqlContext.sql("""
    SELECT duration, dst_bytes FROM interactions WHERE protocol_type = 'tcp' AND duration > 1000 AND dst_bytes = 0
""")
tcp_interactions.show()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 2, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark-2.2.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 123, in main
    ("%d.%d" % sys.version_info[:2], version))
Exception: Python in worker has different version 3.5 than that in driver 3.6, PySpark cannot run with different minor versions.Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.scala:193)
	at org.apache.spark.api.python.PythonRunner$$anon$1.<init>(PythonRDD.scala:234)
	at org.apache.spark.api.python.PythonRunner.compute(PythonRDD.scala:152)
	at org.apache.spark.api

In [ ]:
print('startingggg', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
tcp_interactions.write.format('csv').save("/results/spark-result", sep='|', header=True)
print('endingggg', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
sc.stop()